In [1]:
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")
from ABSE import absec
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109
%load_ext autoreload
%autoreload 2
from AP_class import alch_deriv,first_deriv_nuc_nuc,second_deriv_nuc_nuc
from AP_class import APDFT_perturbator as AP
from AP_symmetry import benz_Symm

In [2]:
benz_atom=""" 
C         6.95191249e-13  2.61173060e+00 -1.67580194e-08 ;
H         2.83052087e-13  4.63941350e+00 -1.15976219e-08 ;
C         2.26182505e+00  1.30586530e+00 -2.37508246e-08 ; 
H         4.01784995e+00  2.31970675e+00 -9.84687205e-09 ; 
C         2.26182505e+00 -1.30586530e+00  8.26065094e-09 ; 
H         4.01784995e+00 -2.31970675e+00  5.83890508e-08 ; 
C        -8.45954456e-13 -2.61173060e+00 -1.55922267e-08 ; 
H         1.49445304e-12 -4.63941350e+00 -3.26774154e-08 ; 
C        -2.26182505e+00 -1.30586530e+00 -8.28266973e-09 ; 
H        -4.01784995e+00 -2.31970675e+00  1.87236306e-09 ; 
C        -2.26182505e+00  1.30586530e+00  1.68546675e-08 ; 
H        -4.01784995e+00  2.31970675e+00  3.31289175e-08 ; 
"""
Benz = gto.M(atom=benz_atom, basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])},unit='bohr')

In [3]:
benz_coords=Benz.atom_coords()

In [4]:
benz_dm0=np.load("benz_dm.npy")
BenzHF=scf.RHF(Benz)
BenzHF.scf(dm0=benz_dm0)

converged SCF energy = -230.788695455662


-230.7886954556621

In [5]:
apbz=AP(BenzHF,sites=[0,2,4,6,8,10],symm=benz_Symm(Benz))

In [6]:
apbz.build_all()

In [7]:
apbz.af(0)

No AF found for atom 0. Calculating it from code


array([[ 6.04841068e-14,  2.15252224e-02, -8.55167900e-10],
       [-7.15775290e-15,  7.83866964e-02,  6.27696648e-11],
       [ 6.76028944e-02, -3.29035758e-02, -9.90819102e-10],
       [ 9.50152738e-03, -5.77177984e-03,  3.70390033e-10],
       [ 2.91178600e-03,  9.40043920e-04, -1.87825670e-10],
       [ 1.70413431e-03, -5.34737950e-03,  2.06301191e-10],
       [ 1.24148164e-14, -1.13075776e-02,  1.81661639e-10],
       [-3.29942158e-15, -2.43895882e-03,  2.29287108e-11],
       [-2.91178600e-03,  9.40043920e-04,  5.64727505e-12],
       [-1.70413431e-03, -5.34737950e-03, -9.18982504e-11],
       [-6.76028944e-02, -3.29035758e-02,  1.20194718e-09],
       [-9.50152738e-03, -5.77177984e-03,  7.40652339e-11]])

In [8]:
apbz.af(2)

array([[-6.22967797e-02,  4.20940361e-02,  1.20194718e-09],
       [-9.74927165e-03,  5.34267417e-03,  7.40652339e-11],
       [ 1.86413894e-02,  1.07626112e-02, -8.55167900e-10],
       [ 6.78848704e-02,  3.91933482e-02,  6.27696648e-11],
       [ 5.30611472e-03, -7.49976118e-02, -9.90819102e-10],
       [-2.47744275e-04, -1.11144540e-02,  3.70390033e-10],
       [ 2.26999491e-03, -2.05165868e-03, -1.87825670e-10],
       [-3.77889933e-03, -4.14951335e-03,  2.06301191e-10],
       [-9.79264947e-03, -5.65378881e-03,  1.81661639e-10],
       [-2.11220030e-03, -1.21947941e-03,  2.29287108e-11],
       [-6.41791082e-04,  2.99170260e-03,  5.64727505e-12],
       [-5.48303364e-03, -1.19786615e-03, -9.18982504e-11]])

In [9]:
mut_list1=["NBCCCC","NCBCCC","NCCBCC"]
mut_list2=["NBNBCC","NBBNCC","BNNBCC","BBNNCC","NBNCBC","BNBCNC","BNNCBC","NBBCNC"]#4-11
mut_list3=["NBCBNC","NNCBBC","NBCNBC","NNNBBB","NBNBNB","NNBBNB"] #12-17

In [10]:
charge2symbol={1:"H",2:"He",3:"Li",4:"Be",5:"B",6:"C",7:"N",8:"O",9:"F",10:"Ne"}
symbol2charge={"H":1,"He":2,"Li":3,"Be":4,"B":5,"C":6,"N":7,"O":8,"F":9,"Ne":10}
af_array=np.asarray([apbz.af(0),apbz.af(2),apbz.af(4),apbz.af(6),apbz.af(8),apbz.af(10)])

In [11]:
np.einsum("ijk,i->jk",af_array,[1,0,1,0,0,0])-apbz.af(0)-apbz.af(4)

array([[ 0.00000000e+00,  8.67361738e-19,  0.00000000e+00],
       [ 0.00000000e+00, -1.73472348e-18,  0.00000000e+00],
       [-3.46944695e-18,  0.00000000e+00,  0.00000000e+00],
       [-8.67361738e-19,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.08420217e-19,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -2.16840434e-19,  0.00000000e+00],
       [ 1.73472348e-18, -8.67361738e-19,  0.00000000e+00],
       [-8.67361738e-19,  2.16840434e-19,  6.46234854e-27]])

In [12]:
[symbol2charge[x]-6 for x in mut_list1[1]]

[1, 0, -1, 0, 0, 0]

In [13]:
h0=np.load("benz_hessian.npy")

In [14]:
apbz.af(2)

array([[-6.22967797e-02,  4.20940361e-02,  1.20194718e-09],
       [-9.74927165e-03,  5.34267417e-03,  7.40652339e-11],
       [ 1.86413894e-02,  1.07626112e-02, -8.55167900e-10],
       [ 6.78848704e-02,  3.91933482e-02,  6.27696648e-11],
       [ 5.30611472e-03, -7.49976118e-02, -9.90819102e-10],
       [-2.47744275e-04, -1.11144540e-02,  3.70390033e-10],
       [ 2.26999491e-03, -2.05165868e-03, -1.87825670e-10],
       [-3.77889933e-03, -4.14951335e-03,  2.06301191e-10],
       [-9.79264947e-03, -5.65378881e-03,  1.81661639e-10],
       [-2.11220030e-03, -1.21947941e-03,  2.29287108e-11],
       [-6.41791082e-04,  2.99170260e-03,  5.64727505e-12],
       [-5.48303364e-03, -1.19786615e-03, -9.18982504e-11]])

In [15]:
def nr_step(ic,g,h):
    ncoords=ic.shape[0]*ic.shape[1]
    p1cds=ic-np.linalg.solve((np.swapaxes(h,1,2)).reshape(ncoords,ncoords),g.reshape(-1)).reshape(ic.shape[0],ic.shape[1])
    #print (norm(p1cds[1]))
    return(p1cds)


In [16]:
def nr_DE(g,h):
    ncoords=g.shape[0]*g.shape[1]
    dr=-np.linalg.solve((np.swapaxes(h,1,2)).reshape(ncoords,ncoords),g.reshape(-1))
    de=0.5*dr.dot(g.reshape(-1))
    return de

In [17]:
nr_step(benz_coords,apbz.af(0)/100,h0)

array([[-1.05461562e-04,  2.61033709e+00,  7.06371362e-09],
       [-1.05458422e-04,  4.63608633e+00,  3.16917095e-08],
       [ 2.26070496e+00,  1.30610973e+00,  2.52142303e-08],
       [ 4.01568658e+00,  2.32154005e+00,  7.79720805e-08],
       [ 2.26166082e+00, -1.30548448e+00,  3.20784199e-08],
       [ 4.01795489e+00, -2.31862501e+00,  1.01626799e-07],
       [-1.05469655e-04, -2.61128192e+00, -4.20275672e-08],
       [-1.05472793e-04, -4.63891618e+00, -7.86163694e-08],
       [-2.26187176e+00, -1.30548447e+00, -5.98251081e-08],
       [-4.01816583e+00, -2.31862499e+00, -8.86087411e-08],
       [-2.26091589e+00,  1.30610973e+00, -9.59950183e-09],
       [-4.01589751e+00,  2.32154006e+00, -1.28122223e-08]])

In [18]:
benz_coords

array([[ 6.95191249e-13,  2.61173060e+00, -1.67580194e-08],
       [ 2.83052087e-13,  4.63941350e+00, -1.15976219e-08],
       [ 2.26182505e+00,  1.30586530e+00, -2.37508246e-08],
       [ 4.01784995e+00,  2.31970675e+00, -9.84687205e-09],
       [ 2.26182505e+00, -1.30586530e+00,  8.26065094e-09],
       [ 4.01784995e+00, -2.31970675e+00,  5.83890508e-08],
       [-8.45954456e-13, -2.61173060e+00, -1.55922267e-08],
       [ 1.49445304e-12, -4.63941350e+00, -3.26774154e-08],
       [-2.26182505e+00, -1.30586530e+00, -8.28266973e-09],
       [-4.01784995e+00, -2.31970675e+00,  1.87236306e-09],
       [-2.26182505e+00,  1.30586530e+00,  1.68546675e-08],
       [-4.01784995e+00,  2.31970675e+00,  3.31289175e-08]])

In [32]:
atomstr="{} {};H {} ;"*6

In [19]:
atomstr

NameError: name 'atomstr' is not defined

In [ ]:
mut=mut_list2[1]
lstfrmt=[]
for i in range(6):
    lstfrmt.append(mut[i])
    lstfrmt.append(str(pg1[2*i])[1:-1])
    lstfrmt.append(str(pg1[2*i+1])[1:-1])
atomstr.format(*lstfrmt)

In [ ]:
molpg1=gto.M(atom=atomstr.format(*lstfrmt),unit="Bohr")
molpg1.atom_coords()

In [ ]:
for idx,mut in enumerate(mut_list1):
    print([symbol2charge[x]-6 for x in mut])
    af=np.einsum("ijk,i->jk",af_array,[symbol2charge[x]-6 for x in mut])
    pg=nr_step(benz_coords,af,h0)
    lstfrmt=[]
    for i in range(6):
        lstfrmt.append(mut[i])
        lstfrmt.append(str(pg[2*i])[1:-1])
        lstfrmt.append(str(pg[2*i+1])[1:-1])
    atomstr.format(*lstfrmt)
    molp=gto.M(atom=atomstr.format(*lstfrmt),unit="Bohr")
    molp.tofile("./NR_H0/{}_NR.xyz".format(idx+1))


In [139]:
for idx,mut in enumerate(mut_list2):
    af=np.einsum("ijk,i->jk",af_array,[symbol2charge[x]-6 for x in mut])
    pg=nr_step(benz_coords,af,h0)
    lstfrmt=[]
    for i in range(6):
        lstfrmt.append(mut[i])
        lstfrmt.append(str(pg[2*i])[1:-1])
        lstfrmt.append(str(pg[2*i+1])[1:-1])
    atomstr.format(*lstfrmt)
    molp=gto.M(atom=atomstr.format(*lstfrmt),unit="Bohr")
    molp.tofile("./NR_H0/{}_NR.xyz".format(idx+4))
    
for idx,mut in enumerate(mut_list3):
    af=np.einsum("ijk,i->jk",af_array,[symbol2charge[x]-6 for x in mut])
    pg=nr_step(benz_coords,af,h0)
    lstfrmt=[]
    for i in range(6):
        lstfrmt.append(mut[i])
        lstfrmt.append(str(pg[2*i])[1:-1])
        lstfrmt.append(str(pg[2*i+1])[1:-1])
    atomstr.format(*lstfrmt)
    molp=gto.M(atom=atomstr.format(*lstfrmt),unit="Bohr")
    molp.tofile("./NR_H0/{}_NR.xyz".format(idx+12))


In [140]:
import rmsd 

In [141]:
for i in range(4,18):
    print(i)
    rmsd.main(args=["./opt/{}_opt.xyz".format(i),"./pred_geoms/{}_p.xyz".format(i)])

4
0.06508065523817591
5
0.13132735437837767
6
0.08640737419107673
7
0.15292434639534427
8
0.07519094224058497
9
0.08390932797615752
10
0.10920925530150724
11
0.15117381124015006
12
0.09472227859300518
13
0.16463786587210355
14
0.07993302928665275
15
0.2131339425317265
16
0.0636595936520292
17
0.14159151060309583


In [145]:
for i in range(1,4):
    print(i)
    rmsd.main(args=["./opt/{}_opt.xyz".format(i),"./NR_H0/{}_NR.xyz".format(i)])

1
0.04022603342277137
2
0.03118568070832377
3
0.04219086509179594


In [142]:
for i in range(4,18):
    print(i)
    rmsd.main(args=["./opt/{}_opt.xyz".format(i),"./NR_H0/{}_NR.xyz".format(i)])
    rmsd.main(args=["./opt/{}_opt.xyz".format(i),"./pred_geoms/{}_p.xyz".format(i)])
    rmsd.main(args=["./opt/{}_opt.xyz".format(i),"./pred_geoms/{}_bg.xyz".format(i)])
    rmsd.main(args=["./NR_H0/{}_NR.xyz".format(i),"./pred_geoms/{}_bg.xyz".format(i)])

4
0.05145613760749521
0.06508065523817591
0.1132974447104944
0.09922193837826412
5
0.0792923344948646
0.13132735437837767
0.1907462624819973
0.1528530765891545
6
0.08154447769207164
0.08640737419107673
0.13339332603821155
0.1528541985851898
7
0.09177090723147235
0.15292434639534427
0.2179715197923148
0.20315574185906582
8
0.0432311765593216
0.07519094224058497
0.1263673603393583
0.12453102269853579
9
0.04612952822585621
0.08390932797615752
0.13952669074949411
0.12452905341430467
10
0.07534621475279375
0.10920925530150724
0.1735515397965274
0.18279908950357465
11
0.06971104758368292
0.15117381124015006
0.2198371540504253
0.18279935546300094
12
0.05646410895937914
0.09472227859300518
0.15876304297141489
0.13710983969022963
13
0.07996004470325228
0.16463786587210355
0.23568516763602587
0.2241110367116062
14
0.04242737086128693
0.07993302928665275
0.12933844027698854
0.12004335936462215
15
0.12965531657203813
0.2131339425317265
0.27815672340721465
0.2597740706839199
16
0.05523962110355735


In [143]:
for i in range(4,18):
    print(i)
    rmsd.main(args=["./pred_geoms/{}_p.xyz".format(i),"./NR_H0/{}_NR.xyz".format(i)])

4
0.03827002468384104
5
0.08672385592944967
6
0.08708471482329563
7
0.12750185569257078
8
0.06454451498357285
9
0.06470099925973614
10
0.10867539332270756
11
0.10834467204514125
12
0.06447342744158287
13
0.1458997833420189
14
0.06483952191219988
15
0.1824334699148727
16
0.021104609553829725
17
0.11329044454932372


# Energies

In [20]:
ve=[]
for m in mut_list1:
    ve.append(apbz.APDFT3([symbol2charge[x]-6 for x in m])+absec("CCCCCC",m))
for m in mut_list2:
    ve.append(apbz.APDFT3([symbol2charge[x]-6 for x in m])+absec("CCCCCC",m))
for m in mut_list3:
    ve.append(apbz.APDFT3([symbol2charge[x]-6 for x in m])+absec("CCCCCC",m))

In [21]:
ve

[-234.19580818106718,
 -234.11935890563151,
 -234.13764309315144,
 -237.6976543697109,
 -237.50709491657636,
 -237.50927997188262,
 -237.39185726800594,
 -237.59195703128017,
 -237.57731640810732,
 -237.46207875907987,
 -237.47453432793156,
 -237.63948928161636,
 -237.33369217990912,
 -237.56635253183032,
 -240.68263981753034,
 -241.29423402092652,
 -240.91530016990535]

In [22]:
der=[]
for m in mut_list1:
    af=np.einsum("ijk,i->jk",af_array,[symbol2charge[x]-6 for x in m])
    der.append(nr_DE(af,h0))
for m in mut_list2:
    af=np.einsum("ijk,i->jk",af_array,[symbol2charge[x]-6 for x in m])
    der.append(nr_DE(af,h0))
for m in mut_list3:
    af=np.einsum("ijk,i->jk",af_array,[symbol2charge[x]-6 for x in m])
    der.append(nr_DE(af,h0))

In [23]:
der

[-0.029960324479483567,
 -0.04740279075787867,
 -0.048841678347085776,
 -0.04391707031711075,
 -0.07592422760433605,
 -0.07592422760433605,
 -0.11368693542937755,
 -0.05848176150159617,
 -0.05848176150159617,
 -0.09624446866380286,
 -0.09624446866380286,
 -0.06279842389538218,
 -0.13256828900309459,
 -0.05704287402040083,
 -0.18140996671125006,
 -0.04187023728838517,
 -0.10588455240152712]

In [24]:
re=[]
for i in range(len(ve)):
    re.append(ve[i]+der[i])

In [25]:
re

[-234.22576850554665,
 -234.1667616963894,
 -234.18648477149853,
 -237.74157144002803,
 -237.5830191441807,
 -237.58520419948695,
 -237.5055442034353,
 -237.65043879278176,
 -237.6357981696089,
 -237.55832322774367,
 -237.57077879659536,
 -237.70228770551174,
 -237.46626046891222,
 -237.62339540585072,
 -240.8640497842416,
 -241.3361042582149,
 -241.02118472230688]

In [13]:
ve2=[]
for m in mut_list1:
    ve2.append(apbz.APDFT2([symbol2charge[x]-6 for x in m])+absec("CCCCCC",m))
for m in mut_list2:
    ve2.append(apbz.APDFT2([symbol2charge[x]-6 for x in m])+absec("CCCCCC",m))
for m in mut_list3:
    ve2.append(apbz.APDFT2([symbol2charge[x]-6 for x in m])+absec("CCCCCC",m))

In [14]:
ve2

[-234.1958081810586,
 -234.11935890563154,
 -234.13764309315025,
 -237.69765436971124,
 -237.5081874437176,
 -237.50818744474554,
 -237.39185726800991,
 -237.58463671943753,
 -237.5846367199536,
 -237.46830654376794,
 -237.46830654324776,
 -237.63948928161614,
 -237.33369217990997,
 -237.56635253181224,
 -240.68263981753205,
 -241.2942340209282,
 -240.91530016990546]

In [25]:
re2=[]
for i in range(len(ve2)):
    re2.append(ve2[i]+der[i])

In [26]:
re2

[-234.22576850553781,
 -234.16676169638913,
 -234.18648477149694,
 -237.74157144002774,
 -237.58411167132147,
 -237.58411167234942,
 -237.5055442034386,
 -237.64311848093868,
 -237.64311848145476,
 -237.56455101243105,
 -237.56455101191088,
 -237.70228770551077,
 -237.46626046891225,
 -237.6233954058323,
 -240.8640497842421,
 -241.33610425821553,
 -241.02118472230626]

In [27]:
np.asarray(ve2)-np.asarray(re2)

array([0.02996032, 0.04740279, 0.04884168, 0.04391707, 0.07592423,
       0.07592423, 0.11368694, 0.05848176, 0.05848176, 0.09624447,
       0.09624447, 0.06279842, 0.13256829, 0.05704287, 0.18140997,
       0.04187024, 0.10588455])

In [34]:
ve2.sort(),re2.sort()

In [37]:
ve2

[-241.2942340209282,
 -240.91530016990546,
 -240.68263981753205,
 -237.69765436971124,
 -237.63948928161614,
 -237.5846367199536,
 -237.58463671943753,
 -237.56635253181224,
 -237.50818744474554,
 -237.5081874437176,
 -237.46830654376794,
 -237.46830654324776,
 -237.39185726800991,
 -237.33369217990997,
 -234.1958081810586,
 -234.13764309315025,
 -234.11935890563154]

In [38]:
re2

[-234.22576850553781,
 -234.16676169638913,
 -234.18648477149694,
 -237.74157144002774,
 -237.58411167132147,
 -237.58411167234942,
 -237.5055442034386,
 -237.64311848093868,
 -237.64311848145476,
 -237.56455101243105,
 -237.56455101191088,
 -237.70228770551077,
 -237.46626046891225,
 -237.6233954058323,
 -240.8640497842421,
 -241.33610425821553,
 -241.02118472230626]